In [3]:
import pandas as pd

df_eta = pd.read_parquet("Data/raw/StockEtablissement_utf8.parquet", engine='pyarrow')

In [4]:
df_eta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42151993 entries, 0 to 42151992
Data columns (total 53 columns):
 #   Column                                          Dtype         
---  ------                                          -----         
 0   siren                                           object        
 1   nic                                             int64         
 2   siret                                           object        
 3   statutDiffusionEtablissement                    object        
 4   dateCreationEtablissement                       object        
 5   trancheEffectifsEtablissement                   object        
 6   anneeEffectifsEtablissement                     float64       
 7   activitePrincipaleRegistreMetiersEtablissement  object        
 8   dateDernierTraitementEtablissement              datetime64[us]
 9   etablissementSiege                              bool          
 10  nombrePeriodesEtablissement                     int64         
 

In [5]:
df_eta.head()

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,...,libellePaysEtranger2Etablissement,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
0,000325175,16,00032517500016,O,2000-09-26,NN,NaN,3212ZZ,2024-03-22 15:40:57,False,...,None,2009-05-27,F,None,None,None,None,32.12Z,NAFRev2,N
1,000325175,24,00032517500024,O,2008-05-20,NN,NaN,None,2025-01-27 02:35:48,False,...,None,2011-10-21,F,TAHITI PERLES CREATIONS,None,None,None,47.89Z,NAFRev2,N
2,000325175,32,00032517500032,O,2009-05-27,NN,NaN,None,2025-01-27 02:31:40,False,...,None,2011-10-21,F,TAHITI PERLES CREATIONS,None,None,None,32.12Z,NAFRev2,N
3,000325175,40,00032517500040,O,2011-10-21,NN,NaN,3212ZZ,2024-03-30 05:48:29,False,...,None,2014-01-07,F,TAHITI PERLES CREATION,None,None,None,32.12Z,NAFRev2,N
4,000325175,57,00032517500057,O,2014-01-07,NN,NaN,None,2024-03-30 05:29:10,False,...,None,2018-02-07,F,TAHITI PERLES CREATION,None,None,None,32.12Z,NAFRev2,N


In [6]:
df_unit_legal = pd.read_parquet("Data/raw/StockUniteLegale_utf8.parquet", engine='pyarrow')

In [7]:
df_unit_legal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28882409 entries, 0 to 28882408
Data columns (total 34 columns):
 #   Column                                     Dtype         
---  ------                                     -----         
 0   siren                                      object        
 1   statutDiffusionUniteLegale                 object        
 2   unitePurgeeUniteLegale                     object        
 3   dateCreationUniteLegale                    object        
 4   sigleUniteLegale                           object        
 5   sexeUniteLegale                            object        
 6   prenom1UniteLegale                         object        
 7   prenom2UniteLegale                         object        
 8   prenom3UniteLegale                         object        
 9   prenom4UniteLegale                         object        
 10  prenomUsuelUniteLegale                     object        
 11  pseudonymeUniteLegale                      object        
 12

In [8]:
df_unit_legal.head()

,siren,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,sigleUniteLegale,sexeUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,...,denominationUsuelle1UniteLegale,denominationUsuelle2UniteLegale,denominationUsuelle3UniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,nomenclatureActivitePrincipaleUniteLegale,nicSiegeUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,caractereEmployeurUniteLegale
0,000325175,O,None,2000-09-26,None,M,THIERRY,None,None,None,...,None,None,None,1000,32.12Z,NAFRev2,65,None,None,None
1,001807254,O,None,1972-05-01,None,M,JACQUES-LUCIEN,None,None,None,...,None,None,None,1000,85.59A,NAFRev2,22,None,None,None
2,005410220,O,True,1954-12-25,None,M,GEORGES,None,None,None,...,None,None,None,1000,22.02,NAP,15,None,None,None
3,005410345,O,True,None,None,M,MICHEL,None,None,None,...,None,None,None,1000,79.06,NAP,10,None,None,None
4,005410394,O,True,1954-12-25,None,M,ROBERT,ALFRED,None,None,...,None,None,None,1000,64.42,NAP,18,None,None,None


In [ ]:
import pandas as pd

df_bilan = pd.read_parquet("Data/raw/ExportDetailBilan.parquet", engine='pyarrow')

In [5]:
import pandas as pd
import polars as pl
import pyarrow.parquet as pq  # C'est le moteur 4x4
import sys

# Indique le chemin de ton fichier
filepath = "../Data/raw/ExportDetailBilan.parquet" 

print("--- Début de la lecture 'bypass' ---")

try:
    # ÉTAPE 1 : On lit avec le 4x4 (PyArrow), qui est robuste.
    # On ne prend que les 2 colonnes qui nous intéressent (c'est plus rapide)
    print(f"Lecture du fichier via PyArrow (le moteur robuste) : {filepath}")
    table_arrow = pq.read_table(
        filepath,
        columns=["siren", "liasse"]
    )
    
    # ÉTAPE 2 : On passe la data (sans la copier !) à la F1 (Polars)
    print("Conversion de la table PyArrow en DataFrame Polars...")
    df_bilan_pl = pl.from_arrow(table_arrow)
    
    print("--- SUCCÈS ! ---")
    print("Le DataFrame est maintenant dans Polars, prêt pour la transformation.")
    print(df_bilan_pl.head())

except Exception as e:
    print(f"--- ERREUR ---", file=sys.stderr)
    print(f"Impossible de lire le fichier, même avec PyArrow : {e}", file=sys.stderr)

--- Début de la lecture 'bypass' ---
Lecture du fichier via PyArrow (le moteur robuste) : ../Data/raw/ExportDetailBilan.parquet
Conversion de la table PyArrow en DataFrame Polars...
--- SUCCÈS ! ---
Le DataFrame est maintenant dans Polars, prêt pour la transformation.
shape: (5, 2)
┌───────────┬─────────────────────────────────┐
│ siren     ┆ liasse                          │
│ ---       ┆ ---                             │
│ str       ┆ list[struct[2]]                 │
╞═══════════╪═════════════════════════════════╡
│ 005420120 ┆ [{"HF",111571}, {"BH-BI",2559}… │
│ 005420120 ┆ [{"GK",74833}, {"AS",90304}, …… │
│ 005420120 ┆ [{"HF",151562}, {"NH",5850813}… │
│ 005420120 ┆ [{"HC",918483}, {"CU-CV",66917… │
│ 005420120 ┆ [{"AO",425420}, {"CH-CI",14383… │
└───────────┴─────────────────────────────────┘


In [6]:
df_bilan_pl.describe()


statistic,siren,liasse
str,str,f64
"""count""","""4976444""",4.976444e6
"""null_count""","""0""",0.0
"""mean""",null,null
"""std""",null,null
"""min""","""005420120""",null
"""25%""",null,null
"""50%""",null,null
"""75%""",null,null
"""max""","""999990542""",null


In [8]:
# Je veuxc montrer tous ce qu'il y a dans la colonne liasse pour la premier ligne
liasse_premiere_ligne = df_bilan_pl.select(pl.col("liasse")).row(0)[0]
liasse_premiere_ligne

[{'key': 'HF', 'value': 111571},
 {'key': 'BH-BI', 'value': 2559},
 {'key': 'CJ-CK', 'value': 15117606},
 {'key': 'DO', 'value': 0},
 {'key': 'EA', 'value': 123502},
 {'key': 'EG', 'value': 841098},
 {'key': 'FW', 'value': 385776},
 {'key': 'GA', 'value': 224479},
 {'key': 'GL', 'value': 619956},
 {'key': 'CD-CE', 'value': 12917722},
 {'key': 'QU', 'value': 2368581},
 {'key': 'CF', 'value': 1404319},
 {'key': 'FY', 'value': 559989},
 {'key': 'AT', 'value': 244629},
 {'key': 'EC', 'value': 4341098},
 {'key': 'FJ', 'value': 135797},
 {'key': 'FL', 'value': 135797},
 {'key': 'UC', 'value': 12390},
 {'key': 'EB', 'value': 338608},
 {'key': 'CD', 'value': 13219183},
 {'key': 'CF-CG', 'value': 1404319},
 {'key': 'FA', 'value': 4623},
 {'key': 'HE', 'value': 0},
 {'key': 'HN', 'value': -289131},
 {'key': 'ØJ', 'value': 2377248},
 {'key': 'PE', 'value': 28475},
 {'key': 'AN-AO', 'value': 1888651},
 {'key': 'LW', 'value': 568768},
 {'key': 'MY', 'value': 0},
 {'key': 'ØR', 'value': 2563605},
 {